In [84]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score

In [85]:
dataframe = pd.read_csv('salida/resultados/SICK/DEV_5.csv.csv')

In [86]:
dataframe

,suma,distancias,entropias,mutual_info,m_earth,max_info_p,antonimos,relaciones,sim_faltantes,CLASS
0,0.000000,91.15190,0.000,1.192824e-12,16.474854,0.000000,0,0,0.000000,CONTRADICTION
1,0.000000,0.00000,0.000,0.000000e+00,0.000000,0.000000,0,1,0.000000,ENTAILMENT
2,0.000000,0.00000,0.000,0.000000e+00,0.000000,0.000000,0,0,1.000000,CONTRADICTION
3,1.185669,209.10780,1.500,7.031703e+00,1.045989,0.296417,1,2,0.487278,CONTRADICTION
4,0.760222,152.18210,1.000,3.512053e+00,1.129299,0.380111,1,0,0.472291,CONTRADICTION
...,...,...,...,...,...,...,...,...,...,...
90,0.986909,156.99320,0.918,1.061669e+01,2.130698,0.199834,0,1,0.287339,CONTRADICTION
91,0.302330,73.98176,1.000,3.260589e+00,0.495572,0.404658,0,0,0.260126,CONTRADICTION
92,1.608502,277.95917,2.459,2.124758e+01,3.813996,0.197763,0,0,0.246701,CONTRADICTION
93,0.595234,170.75844,1.500,6.351086e+00,1.874976,0.205298,0,2,0.162272,CONTRADICTION


In [87]:
X = np.array(dataframe.drop(['CLASS'],axis=1))
y_ = np.array(dataframe['CLASS'])

In [88]:
#SNLI
# y=[]
# for i in y_:
#     if i == "entailment":
#         y.append(1)
#     elif i== "contradiction":
#         y.append(2)
#     else:
#         y.append(0)

In [89]:
# RTE3
# y=[]
# for i in y_:
#     if i == "YES":
#         y.append(1)
#     else:
#         y.append(0)

In [90]:
# SICK
y=[]
for i in y_:
    if i == "ENTAILMENT":
        y.append(1)
    else:
        y.append(0)

In [91]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X = X_scaled

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=7)

In [93]:
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

In [94]:
import xgboost as xgb

In [95]:
# Define hyperparameters
#params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

In [96]:
clf_xgb = xgb.XGBClassifier(objective='reg:logistic',n_estimators=3000,device="cuda")
clf_xgb.fit(X_train, y_train)
preds_xgb = clf_xgb.predict(X_test)

In [97]:
accuracy_score(y_test,preds_xgb)

0.631578947368421

In [98]:
precision_score(y_test,preds_xgb)

0.5

In [99]:
f1_score(y_test,preds_xgb)

0.4615384615384615

In [100]:
recall_score(y_test,preds_xgb)

0.42857142857142855

In [101]:
clf_lr = LR().fit(X_train, y_train)

In [102]:
accuracy_xgb = float(np.sum(preds_xgb == np.array(y_test)))/len(np.array(y_test))
print('Accuracy de XGBoost: ', accuracy_xgb)
accuracy_lr = clf_lr.score(X_test, y_test)
print('Accuracy de Logistic Regression: ', accuracy_lr)

Accuracy de XGBoost:  0.631578947368421
Accuracy de Logistic Regression:  0.7368421052631579


In [103]:
confusion_matrix(y_test, preds_xgb)

array([[9, 3],
       [4, 3]])

In [104]:
data=xgb.DMatrix(X, label=y)

In [105]:
#Performing cross-validation
#cv_results = xgb.cv(data, num_boost_round=10, nfold=3,metrics='logloss', seed=42)

#Printing the results
#print(cv_results)